In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [3]:
test = pd.read_csv('raw/test.csv')
test.head(1)

,ID,shop_id,item_id
0,0,5,5037


In [4]:
sample_submission = pd.read_csv('raw/sample_submission.csv')
sample_submission.head(1)

,ID,item_cnt_month
0,0,0.5


In [5]:
test.count()

ID         214200
shop_id    214200
item_id    214200
dtype: int64

In [6]:
sample_submission.count()

ID                214200
item_cnt_month    214200
dtype: int64

In [19]:
train = pd.read_csv('raw/sales_train.csv')
train['date'] = pd.to_datetime(train['date'])
train.head(2)
# TODO: exclude date leaks

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.0,1.0
1,2013-03-01,0,25,2552,899.0,1.0


In [40]:
def forecast_as_all_time_average(trn, forecast_month_length):
    forecast = (trn
        .groupby(['item_id', 'shop_id'],
                as_index=False)
        .agg({'item_cnt_day': ['sum', 'count'],
              'date': ['min', 'max']})

    )
    forecast['sale_days'] = (forecast['date']['max']-forecast['date']['min']).dt.days + 1
    forecast['avg_daily_sale_items'] = forecast['item_cnt_day']['sum'] / forecast['sale_days']
    # TODO: wouldn't it be better to split daily forecasting and monthly aggs into like separate things to experiment?
    forecast = pd.DataFrame({
        'item_id': forecast['item_id'],
        'shop_id': forecast['shop_id'],
        'item_cnt_month': forecast['avg_daily_sale_items']*forecast_month_length
    })
    return forecast

,item_id,shop_id,item_cnt_month
0,0,54,30.000000
1,1,55,0.645161
2,2,54,0.545455


In [69]:
example = pd.DataFrame(
    columns=['date', 'item_id', 'shop_id', 'item_cnt_day'],
    data=[
        ['2010-01-01', 1, 1, 2],
        ['2010-01-01', 1, 2, 3],
        ['2010-01-01', 2, 1, 1],
        ['2010-01-01', 3, 1, 1],
        ['2010-01-06', 3, 1, 2],
    ]
)
example['date'] = pd.to_datetime(example['date'])

# TODO: wrap that in proper test ids
forecast = forecast_as_all_time_average(example, forecast_month_length=30)
forecast

,item_id,shop_id,item_cnt_month
0,1,1,60.0
1,1,2,90.0
2,2,1,30.0
3,3,1,15.0


In [81]:
assert 60 == forecast.query('item_id == 1 and shop_id == 1')['item_cnt_month'].values[0]
assert 90 == forecast.query('item_id == 1 and shop_id == 2')['item_cnt_month'].values[0]
assert 30 == forecast.query('item_id == 2 and shop_id == 1')['item_cnt_month'].values[0]
assert 15 == forecast.query('item_id == 3 and shop_id == 1')['item_cnt_month'].values[0]

In [82]:
train_forecast = forecast_as_all_time_average(train, 30)

In [111]:
def forecast_to_submission(forecast):
    submission = test.merge(forecast, on=['item_id', 'shop_id'], how='left')
    assert len(test) == len(submission), (f"Generated submission length({len(submission)})" +
     f" doesn't match to expected submission length ({len(test)})")
    submission.fillna(0, inplace=True)
    submission.drop(['shop_id', 'item_id'], axis=1, inplace=True)
    return submission

submission = forecast_to_submission(train_forecast)

In [112]:
!mkdir -p submissions

In [114]:
submission.to_csv('submissions/submission_0001_avg_future_unfiltered.csv', index=False)

In [115]:
# Score: 11.15418
!head -n2 submissions/submission_0001_avg_future_unfiltered.csv

ID,item_cnt_month
0,1.1304347826086958
